<a href="https://colab.research.google.com/github/zjc664656505/BQ_PROJECT/blob/main/BQ_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load data from Google Drive

In [1]:
#Mount to your google drive
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [2]:
#define method for feature selection
def feature_selection(dataframe, feature_list):
  '''
    This is a method which returns the new dataframe in which the features are selected
  '''
  return dataframe[feature_list]

In [61]:
data = pd.read_csv("/content/drive/MyDrive/BrightQuery_Project/meta-model/new_merged_meta_data.csv")

### Transfer data with years as variables

In [80]:
primary_key = ['bq_id', 'bq_year']
other_group_key = ['bq_sector_name', 'bq_business_code', 'bq_company_address1_state']
selecte_features = ['bq_revenue', 'bq_ebitda','bq_operating_expenses','bq_gross_profit', 'bq_cor', 
                    'bq_operating_income', 'bq_tax_and_interest', 'bq_net_income']

# get the data
net_incoome_public = feature_selection(data, primary_key+selecte_features)


In [90]:
# get the company info
categorical_variable = feature_selection(data, ['bq_id']+other_group_key)
company_info = categorical_variable.drop_duplicates(subset=['bq_id'])
# Convert categorical variable into dummy/indicator variables.
company_info = pd.get_dummies(data=company_info)

In [81]:
# Tranfer the long date to wide data
new_pd = net_incoome_public[(net_incoome_public.bq_year == 2015)].drop(columns=['bq_year'])
new_pd.columns = new_pd.columns+"_"+str(2015)
new_pd = new_pd.rename(columns={'bq_id_'+str(2015): 'bq_id'})

for i in range(2016, 2021):
  next_year_net_income = net_incoome_public[(net_incoome_public.bq_year == i)].drop(columns=['bq_year'])
  next_year_net_income.columns = next_year_net_income.columns+"_"+str(i)
  next_year_net_income = next_year_net_income.rename(columns={'bq_id_'+str(i): 'bq_id'})
  new_pd = new_pd.merge(next_year_net_income, how="outer", on='bq_id')

new_pd

,bq_id,bq_revenue_2015,bq_ebitda_2015,bq_operating_expenses_2015,bq_gross_profit_2015,bq_cor_2015,bq_operating_income_2015,bq_tax_and_interest_2015,bq_net_income_2015,bq_revenue_2016,bq_ebitda_2016,bq_operating_expenses_2016,bq_gross_profit_2016,bq_cor_2016,bq_operating_income_2016,bq_tax_and_interest_2016,bq_net_income_2016,bq_revenue_2017,bq_ebitda_2017,bq_operating_expenses_2017,bq_gross_profit_2017,bq_cor_2017,bq_operating_income_2017,bq_tax_and_interest_2017,bq_net_income_2017,bq_revenue_2018,bq_ebitda_2018,bq_operating_expenses_2018,bq_gross_profit_2018,bq_cor_2018,bq_operating_income_2018,bq_tax_and_interest_2018,bq_net_income_2018,bq_revenue_2019,bq_ebitda_2019,bq_operating_expenses_2019,bq_gross_profit_2019,bq_cor_2019,bq_operating_income_2019,bq_tax_and_interest_2019,bq_net_income_2019,bq_revenue_2020,bq_ebitda_2020,bq_operating_expenses_2020,bq_gross_profit_2020,bq_cor_2020,bq_operating_income_2020,bq_tax_and_interest_2020,bq_net_income_2020
0,412416919,459244000.0,115001000.0,42079000.0,119837000.0,339407000.0,77758000.0,36618000.0,41140000.0,498860000.0,123148000.0,46759000.0,132742000.0,366118000.0,85983000.0,41308000.0,44675000.0,449646000.0,130045000.0,53077000.0,142807000.0,306839000.0,89730000.0,31606000.0,58124000.0,490316000.0,143295000.0,58393000.0,153236000.0,337080000.0,94843000.0,38263000.0,56580000.0,479604000.0,163120000.0,65294000.0,171581000.0,308023000.0,106287000.0,41134000.0,65153000.0,483154000.0,178694000.0,75221000.0,182360000.0,300794000.0,107139000.0,35428000.0,71711000.0
1,136315656,308733000.0,150819000.0,153384000.0,308733000.0,0.0,155349000.0,63091000.0,92258000.0,406174000.0,191199000.0,214975000.0,406174000.0,0.0,191199000.0,77452000.0,113747000.0,530435000.0,263834000.0,266601000.0,530435000.0,0.0,263834000.0,124389000.0,139445000.0,533207000.0,255481000.0,277726000.0,533207000.0,0.0,255481000.0,65892000.0,189589000.0,508978000.0,226350000.0,282628000.0,508978000.0,0.0,226350000.0,55310000.0,171040000.0,446746000.0,163741000.0,275545000.0,446746000.0,0.0,171201000.0,44996000.0,126205000.0
2,666958061,145863000.0,12639000.0,97487000.0,106428000.0,39435000.0,8941000.0,3551000.0,5390000.0,150582000.0,11097000.0,103213000.0,111037000.0,39545000.0,7824000.0,3794000.0,4030000.0,153877000.0,10592000.0,106909000.0,112280000.0,41597000.0,5371000.0,1332000.0,4039000.0,159990000.0,9745000.0,111922000.0,116954000.0,43036000.0,5032000.0,377000.0,4655000.0,162354000.0,8755000.0,115673000.0,118919000.0,43435000.0,3246000.0,570000.0,2676000.0,106490000.0,-3012000.0,85703000.0,77907000.0,28583000.0,-7796000.0,-3108000.0,-4688000.0
3,105749915,38394000.0,11415000.0,28418000.0,38394000.0,0.0,9976000.0,3633000.0,6343000.0,41342000.0,13291000.0,29595000.0,41342000.0,0.0,11747000.0,4416000.0,7331000.0,46419000.0,17963000.0,30054000.0,46419000.0,0.0,16365000.0,9157000.0,7208000.0,53550000.0,17864000.0,38149000.0,53550000.0,0.0,15401000.0,4173000.0,11228000.0,57170000.0,23262000.0,36233000.0,57170000.0,0.0,20937000.0,5665000.0,15272000.0,56012000.0,20592000.0,38019000.0,56012000.0,0.0,17993000.0,3921000.0,14072000.0
4,218284269,24787824.0,9096537.0,16145736.0,24787824.0,0.0,8642088.0,2925321.0,5716767.0,24707521.0,8451996.0,17100746.0,24707521.0,0.0,7606775.0,2012764.0,5594011.0,27331499.0,8884055.0,19015658.0,27331499.0,0.0,8315841.0,3158177.0,5157664.0,38645025.0,11189237.0,29458333.0,38645025.0,0.0,9186692.0,1854813.0,7331879.0,38596382.0,13545690.0,27498134.0,38596382.0,0.0,11098248.0,1683158.0,9415090.0,38563038.0,12383221.0,28661845.0,38563038.0,0.0,9901193.0,1699504.0,8201689.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353,949778563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45168000.0,-10562000.0,12244000.0,-4254000.0,49422000.0,

In [94]:
df = new_pd.dropna()
drop_columns = (net_incoome_public.columns[2:] + '_'+str(2020)).tolist()
X = df.drop(columns=drop_columns)
y = df['bq_net_income_2020']
# merge X with categorical data
X = X.merge(company_info, how="inner", on='bq_id')


In [101]:
X['bq_id'] = X['bq_id'].astype("category")
X['bq_business_code'] = X['bq_business_code'].astype("category")
X

,bq_id,bq_revenue_2015,bq_ebitda_2015,bq_operating_expenses_2015,bq_gross_profit_2015,bq_cor_2015,bq_operating_income_2015,bq_tax_and_interest_2015,bq_net_income_2015,bq_revenue_2016,bq_ebitda_2016,bq_operating_expenses_2016,bq_gross_profit_2016,bq_cor_2016,bq_operating_income_2016,bq_tax_and_interest_2016,bq_net_income_2016,bq_revenue_2017,bq_ebitda_2017,bq_operating_expenses_2017,bq_gross_profit_2017,bq_cor_2017,bq_operating_income_2017,bq_tax_and_interest_2017,bq_net_income_2017,bq_revenue_2018,bq_ebitda_2018,bq_operating_expenses_2018,bq_gross_profit_2018,bq_cor_2018,bq_operating_income_2018,bq_tax_and_interest_2018,bq_net_income_2018,bq_revenue_2019,bq_ebitda_2019,bq_operating_expenses_2019,bq_gross_profit_2019,bq_cor_2019,bq_operating_income_2019,bq_tax_and_interest_2019,...,bq_company_address1_state_IA,bq_company_address1_state_ID,bq_company_address1_state_IL,bq_company_address1_state_IN,bq_company_address1_state_KS,bq_company_address1_state_KY,bq_company_address1_state_LA,bq_company_address1_state_MA,bq_company_address1_state_MD,bq_company_address1_state_ME,bq_company_address1_state_MI,bq_company_address1_state_MN,bq_company_address1_state_MO,bq_company_address1_state_MS,bq_company_address1_state_MT,bq_company_address1_state_NC,bq_company_address1_state_ND,bq_company_address1_state_NE,bq_company_address1_state_NH,bq_company_address1_state_NJ,bq_company_address1_state_NM,bq_company_address1_state_NV,bq_company_address1_state_NY,bq_company_address1_state_OH,bq_company_address1_state_OK,bq_company_address1_state_OR,bq_company_address1_state_PA,bq_company_address1_state_PR,bq_company_address1_state_RI,bq_company_address1_state_SC,bq_company_address1_state_SD,bq_company_address1_state_TN,bq_company_address1_state_TX,bq_company_address1_state_UT,bq_company_address1_state_VA,bq_company_address1_state_VI,bq_company_address1_state_VT,bq_company_address1_state_WA,bq_company_address1_state_WI,bq_company_address1_state_WV
0,412416919,4.592440e+08,115001000.0,4.207900e+07,1.198370e+08,3.394070e+08,77758000.0,36618000.0,41140000.0,4.988600e+08,1.231480e+08,4.675900e+07,1.327420e+08,3.661180e+08,8.598300e+07,41308000.0,44675000.0,4.496460e+08,130045000.0,5.307700e+07,1.428070e+08,3.068390e+08,89730000.0,31606000.0,58124000.0,4.903160e+08,143295000.0,5.839300e+07,1.532360e+08,3.370800e+08,94843000.0,38263000.0,56580000.0,4.796040e+08,163120000.0,6.529400e+07,1.715810e+08,3.080230e+08,106287000.0,41134000.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,136315656,3.087330e+08,150819000.0,1.533840e+08,3.087330e+08,0.000000e+00,155349000.0,63091000.0,92258000.0,4.061740e+08,1.911990e+08,2.149750e+08,4.061740e+08,0.000000e+00,1.911990e+08,77452000.0,113747000.0,5.304350e+08,263834000.0,2.666010e+08,5.304350e+08,0.000000e+00,263834000.0,124389000.0,139445000.0,5.332070e+08,255481000.0,2.777260e+08,5.332070e+08,0.000000e+00,255481000.0,65892000.0,189589000.0,5.089780e+08,226350000.0,2.826280e+08,5.089780e+08,0.000000e+00,226350000.0,55310000.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,666958061,1.458630e+08,12639000.0,9.748700e+07,1.064280e+08,3.943500e+07,8941000.0,3551000.0,5390000.0,1.505820e+08,1.109700e+07,1.032130e+08,1.110370e+08,3.954500e+07,7.824000e+06,3794000.0,4030000.0,1.538770e+08,10592000.0,1.069090e+08,1.122800e+08,4.159700e+07,5371000.0,1332000.0,4039000.0,1.599900e+08,9745000.0,1.119220e+08,1.169540e+08,4.303600e+07,5032000.0,377000.0,4655000.0,1.623540e+08,8755000.0,1.156730e+08,1.189190e+08,4.343500e+07,3246000.0,570000.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,105749915,3.839400e+07,11415000.0,2.841800e+07,3.839400e+07,0.000000e+00,9976000.0,3633000.0,6343000.0,4.134200e+07,1.329100e+07,2.959500e+07,4.134200e+07,0.000000e+00,1.174700e+07,4416000.0,7331000.0,4.641900e+07,17963000.0,3.005400e+07,4.641900e+07,0.000000e+00,16365000.0,9157000.0,7208000.0,5.355000e+07,17864000.0,3.814900e+07,5.35500

### Model training and results

In [8]:
# define method for Relative Absolute Error (RAE)
def RAE(y_true, y_pred):
  true = np.array(y_true)
  pred = np.array(y_pred)
  numerator =  np.sum(np.abs(pred - true))
  denominator = np.sum(np.abs(true - np.mean(true)))
  return numerator/denominator

# define method for Mean Absolute Error (MAE)
def MAE(y_true, y_pred):
  return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))

In [103]:
# Split the data into training/testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)
r_sq = model.score(X_test, y_test)
print('coefficient of determination:', r_sq)

from sklearn.metrics import mean_squared_error
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
RAE_train_error = RAE(y_train, y_train_pred)
RAE_test_error = RAE(y_test, y_test_pred)
print('RAE_train_error:', RAE_train_error)
print('RAE_test_error', RAE_test_error)

coefficient of determination: 0.8600881675938027
RAE_train_error: 0.4826919906063113
RAE_test_error 0.4384829783149871
